In [1]:
import os
import random

import pandas as pd
import haven.db as db 
from mirrorverse.plotting import plot_h3_slider, plot_h3_animation

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [2]:
sql = '''
with neighbors as (
    select 
        *,
        row_number() over () as _choice
    from 
        movement_model_neighbors
    where
        max_km = 100
)
select 
    n.current_h3_index,
    n.selected_h3_index,
    n.neighbor_h3_index,
    n.date,
    n._individual,
    n._decision,
    n._choice,
    n.tag_key,
    p.mixed_layer_thickness,
    p.salinity,
    p.temperature,
    p.velocity_east,
    p.velocity_north,
    c.chlorophyll,
    c.nitrate,
    c.phosphate,
    c.silicate,
    c.oxygen,
    c.net_primary_production
from 
    neighbors n 
    inner join copernicus_physics p 
        on p.depth_bin = 25 
        and p.region = 'chinook_study'
        and n.neighbor_h3_index = p.h3_index 
        and n.date = p.date 
    inner join copernicus_biochemistry c
        on c.depth_bin = 25 
        and c.region = 'chinook_study'
        and n.neighbor_h3_index = c.h3_index 
        and n.date = c.date 
'''

cached_file_path = 'cached.snappy.parquet'
if os.path.exists(cached_file_path):
    data = pd.read_parquet(cached_file_path)
else:
    data = db.read_data(sql)
data.to_parquet(cached_file_path)

In [3]:
def set_line_color(row):
    if row['neighbor_h3_index'] == row['current_h3_index']:
        return "orange"
    elif row['neighbor_h3_index'] == row['selected_h3_index']:
        return "purple"
    else:
        return "black"
    
data['color'] = data.apply(set_line_color, axis=1)

In [ ]:
data.columns

In [ ]:
individual_key = random.randint(data['_individual'].min(), data['_individual'].max())
print(individual_key)

In [ ]:
value_col = 'temperature'
individual = data[data['_individual'] == individual_key]
fig = plot_h3_animation(
    individual, 
    value_col=value_col,
    h3_col='neighbor_h3_index',
    slider_col='date',
    line_color_col='color',
    bold_colors=['orange', 'purple'],
    zoom=3, 
    center={"lat": 55, "lon": -165}
)
fig.show()

In [14]:
fig.write_html(f"{value_col}_{individual_key}.html")